In [3]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import pandas as pd
import openai


### Read the sampled dataset with Amazon inventory data

In [10]:
df_items = pd.read_json("../../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines =True)

In [11]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,[],4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN
2,All Electronics,USB C Docking Station Dual Monitor for MacBook...,3.9,1193,[【18-in-1Docking Station】With USB C Docking St...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],ZMUIPNG,"[Electronics, Computers & Accessories, Laptop ...","{'Product Dimensions': '3.94""L x 1.18""W x 3.94...",B09SFN9NRX,NaN,NaN,NaN
3,Camera & Photo,[2023 Upgraded] Telescopes for Adults Astronom...,4.1,219,[🎁【2023 All New Experience】The newly upgraded ...,[],169.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Good picture quality', 'url': 'htt...",HUTACT,"[Electronics, Camera & Photo, Binoculars & Sco...","{'Product Dimensions': '32.5""D x 5.5""W x 9.7""H...",B09TP3SZ7C,NaN,NaN,NaN
4,AMAZON FASHION,"Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",4.5,222,"[Leather,Mesh, Imported, Multi-pockets and Lar...",[],24.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],KPIQIU,"[Electronics, Computers & Accessories, Laptop ...",{'Product Dimensions': '16 x 2 x 12 inches; 1....,B0B5H7T7XZ,NaN,NaN,NaN


In [12]:
list(df_items["features"].items())[0]

(0,
 ['【Fast Charging Cord】These USB C cables provide up to a 3A charging current to greatly shorten the charging time, meets QC2.0 /3.0 fast charging protocol,Incredibly charge your phone from 0 to 80% in 50 minute. 480Mbps (40-60M/s) ultra fast data transmission, which leads to a faster data sync.(Note:Cables support fast charging,but require a USB-A QC3.0/QC2.0/AFC charger)',
  '【Universal Compatibility】The USB C Charger Cable is compatible with Samsung Galaxy S20 / S10 / S9 / S8+ / S8 / A02s / A03s,A12 A20 A21 A22 A23 A31 A32 A33 A41 A42 A50 A52 A52s 5G A71 A72 A73,M11 M21 M31 M51,M12 M22 M32 M52,iPad Pro 2018 / 2020,Sony Xperia XZ/X Compact/L1 / XZs / XA1 / X Premium, HTC 10 LG G5 G6,OnePlus 5T / 6T, Lumia 950 / 950XL,Huawei P9 P9 Plus P10 P10 Plus Honor Mate 9 Mate 9 pro Mate 10 pro Mate 10 lite and more',
  '【Premium Workmanship】Unique increased friction design allows you to easily unplug the cable from your charger,combine 250d bulletproof fiber core to build a cable so durable

In [14]:
list(df_items["images"].items())[0]

(0,
 [{'thumb': 'https://m.media-amazon.com/images/I/51G07yWoOBL._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/51G07yWoOBL.jpg',
   'variant': 'MAIN',
   'hi_res': 'https://m.media-amazon.com/images/I/611AVJMH+JL._SL1200_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/41c+40oKkKL._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/41c+40oKkKL.jpg',
   'variant': 'PT01',
   'hi_res': 'https://m.media-amazon.com/images/I/61ihhPW7VCL._SL1200_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/51y1YnwiUZL._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/51y1YnwiUZL.jpg',
   'variant': 'PT02',
   'hi_res': 'https://m.media-amazon.com/images/I/61UkcVETKcL._SL1200_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/41Nvr++q39L._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/41Nvr++q39L.jpg',
   'variant': 'PT03',
   'hi_res': 'https://m.media-amazon.c

### Preprocess title nd features

In [15]:
def preprocess_description(row):
    return f"{row['title']} {' '.join(row['features'])}"

In [16]:
def extract_first_large_image(row):
    return row["images"][0].get("large","")

In [17]:
df_items["description"]=df_items.apply(preprocess_description, axis=1)
df_items["image"]=df_items.apply(extract_first_large_image,axis=1)

In [18]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,image
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN,https://m.media-amazon.com/images/I/51G07yWoOB...
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41bOA5-ogW...
2,All Electronics,USB C Docking Station Dual Monitor for MacBook...,3.9,1193,[【18-in-1Docking Station】With USB C Docking St...,USB C Docking Station Dual Monitor for MacBook...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],ZMUIPNG,"[Electronics, Computers & Accessories, Laptop ...","{'Product Dimensions': '3.94""L x 1.18""W x 3.94...",B09SFN9NRX,NaN,NaN,NaN,https://m.media-amazon.com/images/I/416IzmVKiC...
3,Camera & Photo,[2023 Upgraded] Telescopes for Adults Astronom...,4.1,219,[🎁【2023 All New Experience】The newly upgraded ...,[2023 Upgraded] Telescopes for Adults Astronom...,169.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Good picture quality', 'url': 'htt...",HUTACT,"[Electronics, Camera & Photo, Binoculars & Sco...","{'Product Dimensions': '32.5""D x 5.5""W x 9.7""H...",B09TP3SZ7C,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41wO4J3TT0...
4,AMAZON FASHION,"Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",4.5,222,"[Leather,Mesh, Imported, Multi-pockets and Lar...","Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",24.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],KPIQIU,"[Electronics, Computers & Accessories, Laptop ...",{'Product Dimensions': '16 x 2 x 12 inches; 1....,B0B5H7T7XZ,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41mwlYqT5p...


In [19]:
list(df_items["description"].items())[0]

(0,
 "RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB Type C Fast Charging Cord - Nylon Braided USB C Charger Cable for Galaxy A20/A50/S10/S9/S8+/S8, iPad Pro 2018, Sony XZ, HTC 10, OnePlus 5T, Huawei P9 etc. 【Fast Charging Cord】These USB C cables provide up to a 3A charging current to greatly shorten the charging time, meets QC2.0 /3.0 fast charging protocol,Incredibly charge your phone from 0 to 80% in 50 minute. 480Mbps (40-60M/s) ultra fast data transmission, which leads to a faster data sync.(Note:Cables support fast charging,but require a USB-A QC3.0/QC2.0/AFC charger) 【Universal Compatibility】The USB C Charger Cable is compatible with Samsung Galaxy S20 / S10 / S9 / S8+ / S8 / A02s / A03s,A12 A20 A21 A22 A23 A31 A32 A33 A41 A42 A50 A52 A52s 5G A71 A72 A73,M11 M21 M31 M51,M12 M22 M32 M52,iPad Pro 2018 / 2020,Sony Xperia XZ/X Compact/L1 / XZs / XA1 / X Premium, HTC 10 LG G5 G6,OnePlus 5T / 6T, Lumia 950 / 950XL,Huawei P9 P9 Plus P10 P10 Plus Honor Mate 9 Mate 9 pro Mate 10 pro Mate 1

### Sample 50 items from the dataset

In [20]:
df_sample=df_items.sample(50, random_state=42)

In [21]:
len(df_sample)

50

In [22]:
data_to_embed=df_sample[["description","image","rating_number","price","average_rating","parent_asin"]].to_dict(orient="records")

In [23]:
data_to_embed

[{'description': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil [Native Palm Rejection]- 

### Define the embedding function

In [24]:
response= openai.embeddings.create(
    input="random text",
    model="text-embedding-3-small"
)

In [25]:
len(response.data[0].embedding)

1536

In [7]:
def get_embedding(text, model="text-embedding-3-small"):
    response= openai.embeddings.create(
    input="random text",
    model="text-embedding-3-small"
    )
    return response.data[0].embedding


In [30]:
get_embedding("ai engineering")

[-0.005315877962857485,
 0.005709205288439989,
 0.0045053851790726185,
 0.0017312359996140003,
 -0.03734622895717621,
 -0.0744381844997406,
 -0.017544781789183617,
 0.005081470590084791,
 -0.0231228768825531,
 -0.005951558239758015,
 0.005907855462282896,
 0.01029802393168211,
 -0.02026231586933136,
 0.031879376620054245,
 0.030766934156417847,
 0.0010756904957816005,
 -0.021978652104735374,
 -0.02940022200345993,
 -0.05320645496249199,
 0.025522571057081223,
 0.017481213435530663,
 0.027111772447824478,
 -0.01827581413090229,
 -0.0020381505601108074,
 -0.0050059836357831955,
 0.01265798695385456,
 0.017767269164323807,
 0.011458140797913074,
 0.044434063136577606,
 -0.02545900270342827,
 0.0008184385951608419,
 -0.04491082578897476,
 0.029034705832600594,
 -0.02854205295443535,
 0.008335360325872898,
 -0.004823225550353527,
 0.023869801312685013,
 0.007584462407976389,
 0.03400890529155731,
 -0.00570125924423337,
 -0.005979369394481182,
 -0.03547096997499466,
 0.044656552374362946,
 0

### Create Qdrant collection

In [4]:
qdrant_client=QdrantClient(url="http://localhost:6333")

In [5]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),

)

True

### Embed data

### Test

In [8]:
pointstruct = PointStruct(
    id=0,
    vector=get_embedding("test"),
    payload={
        "text": "test",
        "model":"text-embedding-3-small"
    }
)

In [9]:
pointstruct

PointStruct(id=0, vector=[-0.005331182852387428, 0.005700631532818079, 0.004485026467591524, 0.001735017984174192, -0.03731033578515053, -0.07442998886108398, -0.017590519040822983, 0.005108719225972891, -0.023104440420866013, -0.0058992598205804825, 0.005883369594812393, 0.010304834693670273, -0.02029186487197876, 0.031875863671302795, 0.030763547867536545, 0.0010279013076797128, -0.021960342302918434, -0.029412874951958656, -0.053264155983924866, 0.02555154077708721, 0.01746339723467827, 0.02714056707918644, -0.01828969083726406, -0.002059775171801448, -0.005037213210016489, 0.012656593695282936, 0.017812984064221382, 0.01147276908159256, 0.044460952281951904, -0.025503870099782944, 0.0008297695894725621, -0.044905878603458405, 0.029031507670879364, -0.028538910672068596, 0.008358277380466461, -0.00484653003513813, 0.023867173120379448, 0.007603490259498358, 0.03400516137480736, -0.0056370701640844345, -0.005966793280094862, -0.03546706587076187, 0.044651634991168976, 0.0457957349717

### Amazon data

In [26]:
pointstructs=[]
for i, data in enumerate(data_to_embed):
    embedding=get_embedding(data["description"])
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embedding,
            payload=data,
        )
    )

In [27]:
pointstructs

[PointStruct(id=0, vector=[-0.005331182852387428, 0.005700631532818079, 0.004485026467591524, 0.001735017984174192, -0.03731033578515053, -0.07442998886108398, -0.017590519040822983, 0.005108719225972891, -0.023104440420866013, -0.0058992598205804825, 0.005883369594812393, 0.010304834693670273, -0.02029186487197876, 0.031875863671302795, 0.030763547867536545, 0.0010279013076797128, -0.021960342302918434, -0.029412874951958656, -0.053264155983924866, 0.02555154077708721, 0.01746339723467827, 0.02714056707918644, -0.01828969083726406, -0.002059775171801448, -0.005037213210016489, 0.012656593695282936, 0.017812984064221382, 0.01147276908159256, 0.044460952281951904, -0.025503870099782944, 0.0008297695894725621, -0.044905878603458405, 0.029031507670879364, -0.028538910672068596, 0.008358277380466461, -0.00484653003513813, 0.023867173120379448, 0.007603490259498358, 0.03400516137480736, -0.0056370701640844345, -0.005966793280094862, -0.03546706587076187, 0.044651634991168976, 0.045795734971

In [28]:
len(pointstructs)

50

### Write data to Qdrant

In [29]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-00",
    wait=True,
    points=pointstructs,
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

### Define a function for retrieval

In [30]:
def retrieve_data(query, k=5):
    query_embedding=get_embedding(query)
    results=qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )
    return results

### test retrievall

In [31]:
retrieve_data("what kind of charging cords do you offer?", k=10).points

[ScoredPoint(id=35, version=1, score=0.99999994, payload={'description': 'Gbivbe Photo Storage Bag 4x6, 10 Inner Large Photo Storage Box Photo Cases Store up to 1000 Photos, Photo Organizer Cards Craft Keeper with Handle for Photo Puzzles Cards Seed Packets (White) ', 'image': 'https://m.media-amazon.com/images/I/419hnN5TlFL._AC_.jpg', 'rating_number': 185, 'price': 19.99, 'average_rating': 4.6, 'parent_asin': 'B09ZPV8WBV'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=32, version=1, score=0.99999994, payload={'description': "Fintie 3 Packs Screen Protector Case Compatible with Garmin Venu Sq 2 / Sq 2 Music/Venu Sq/Venu Sq Music, Soft TPU Plated Bumper Full Cover Protective Cases [Scratch-Proof], Black/Clear/Rose Gold Perfect protective case with screen protector only compatible with Garmin Venu Sq 2 / Sq 2 Music / Venu Sq / Venu Sq Music Watch. 360 degree full protection, keep your watch away from scratches. 99% high transparency, provides a better viewing experienc